# <a id='top'></a> Creating a 3D Print
In this notebook we will use everything we've learned to create an STL file to send to our 3D printer.<br>
<br>
*The methodology works, but unfortunately the model created was not able to be printed. Read on for more details.*

---

## Prerequisites
[1) Setting Up the API Connection](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/01SettingUptheAPIConnection.ipynb)<br>
[2) Using the Spotify API](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/02UsingtheSpotifyAPI.ipynb)<br>
[3) Spotify Audio Data Insights](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/03SpotifyAudioDataInsights.ipynb)<br>
[4) Creating a 3D Audio Model](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/04Creatinga3DAudioModel.ipynb)


---

## Table of Contents
* [Libraries](#Librarires)
* [Functions](#Functions)
  * [Connect to Spotify](#ConnectToSpotify)
  * [Find Track URI](#FindURI)
  * [Create Timbre Data Frame](#CreateTimbreDF)
* [Create STL File](#CreateSTL)
  1. [Build Data Frame](#BuildDF)
  1. [Set X, Y, & Z](#SetXYZ)
  1. [Extract STL File](#ExtractSTL)
* [Printing the STL](#PrintSTL)

---

## <a id='Libraries'></a>Libraries

In [1]:
import math
import spotipy
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from spotipy.oauth2 import SpotifyClientCredentials

---

## <a id='Functions'></a> Functions
We will condense everything we learned in our first 4 notebooks into three, convenient functions.

### <a id='ConnectToAPID'></a> Connect to Spotify API
From [Setting Up the API Connection](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/01SettingUptheAPIConnection.ipynb).

In [2]:
def connect_to_spotify():
    sp_client_id = input("Enter your Spotify Client ID: ")
    sp_client_secret = input("Enter your Spotify Secret Token: ")
    
    credentials = SpotifyClientCredentials(client_id = sp_client_id,
                                           client_secret = sp_client_secret)
    
    sp = spotipy.Spotify(client_credentials_manager = credentials)
    
    print("Connected to Spotify!")

    return sp

### <a href='FindURI'></a> Find Track URI
From [Using the Spotify API](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/02UsingtheSpotifyAPI.ipynb).

In [3]:
def get_uri(connection):

    print('\nRetrieving Track URI.')

    artist = input("Input artist name: ")
    track = input("Input track name: ")

    search_term = f'artist:{artist} track:{track}'

    result = connection.search(search_term, type = 'track')['tracks']['items'][0]

    # If no result, return to user
    if not result:
        return print("Search returned no result")
    else:
        print("\nTrack found!")

    track_uri = result['uri']

    artist_name = result['artists'][0]['name']

    track_name = result['name']

    print(f'\nTrack name: {track_name}')
    print(f'Artist name: {artist_name}')
    print(f'Track URI: {track_uri}')

    return track_uri

### <a id='CreateTimbreDF'></a> Create Timbre DataFrame
From [Creating a 3D Audio Model](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/04Creatinga3DAudioModel.ipynb).

In [4]:
def create_timbre_df(uri):

   # Use the track URI to get the audio analysis. We only need the 'segments' portion
    segments = sp.audio_analysis(uri)['segments']

    # Convert it to a dataframe
    segment_df = pd.DataFrame(segments)

    # Create the timbre dataframe by selecting only the columns we need: start, loudness_start, timbre
    timbre_df = segment_df[['start', 'loudness_start', 'timbre']].copy()

    # Rename loudness and start columns
    timbre_df = timbre_df.rename(columns = {'loudness_start': 'loudness', 'start': 'start_time'})

    # Expand timbres
    for i in range(12):
        timbre_df[f'timbre_{i + 1}'] = timbre_df['timbre'].apply(lambda x: x[i])

    # Drop the original timbre column
    timbre_df.drop(['timbre'], axis = 1, inplace = True)

    # Add '13th timbre'
    timbre_df['timbre_13'] = timbre_df.timbre_1

    # Create a list of the timbre columns
    timbre_cols = [col for col in timbre_df.columns if col.startswith('timbre_')]
    
    # Melt the timbres
    timbre_df = timbre_df.melt(id_vars = ['start_time', 'loudness'],
                               value_vars = timbre_cols,
                               var_name = 'timbre_which',
                               value_name = 'timbre')
    
    # Create the 'timbre_num' column by removing the the word 'timbre' in the 'timbre_which' column
    timbre_df['timbre_num'] = [int(timbre_which.split('_')[1]) for timbre_which in timbre_df['timbre_which']]
    
    # Drop the timbre_which column
    timbre_df.drop(['timbre_which'], axis = 1, inplace = True)
    
    # Sort the dataframe by start_time and timbre_num
    timbre_df = timbre_df.sort_values(by = ['start_time', 'timbre_num']).reset_index(drop = True)

    # Apply the log transform
    timbre_df['timbre_log'] = np.log10(timbre_df['timbre'] + 400)

    # Apply the MinMax scaler to the timbre column
    timbre_df['timbre_scaled'] = MinMaxScaler().fit_transform(timbre_df['timbre'].values.reshape(-1, 1))

    # Adjust values to bounds [1, 2]
    timbre_df['timbre_scaled'] = timbre_df['timbre_scaled'] + 1

    # Apply MinMax scaler to the loudness column. 
    timbre_df['loudness_scaled'] = MinMaxScaler().fit_transform(timbre_df['loudness'].values.reshape(-1, 1))

    # Adjust values to bounds [1, 2]
    timbre_df['loudness_scaled'] = timbre_df['loudness_scaled'] + 1

    #  Scale timbre_log with loudness
    timbre_df['timbre_log_w_loud'] = timbre_df['loudness_scaled'] * timbre_df['timbre_log']
    
    # Scale timbre_scaled with loudness
    timbre_df['timbre_scaled_w_loud'] = timbre_df['loudness_scaled'] * timbre_df['timbre_scaled']

    # Create a column of angles based on timbre number (timbre_num)
    timbre_df['timbre_angle'] = 30 * timbre_df['timbre_num']
    
    # The sin and cos functions in the math package will only take the angles as radians. 
    # Convert the column just created to radians
    timbre_df['timbre_angle'] = (math.pi * timbre_df['timbre_angle']) / 180
    
    # Create timbre_X
    # For this, we want to use the timbre_log_w_loud variable
    timbre_df['timbre_X'] = round(timbre_df['timbre_log_w_loud'] * timbre_df['timbre_angle'].apply(lambda z: math.cos(z)), 3)
    
    # Create timbre_Y using timbre_log_w_loud
    timbre_df['timbre_Y'] = round(timbre_df['timbre_log_w_loud'] * timbre_df['timbre_angle'].apply(lambda z: math.sin(z)), 3)
    
    # We don't need the angle column anymore
    timbre_df.drop(['timbre_angle'], axis = 1, inplace = True)

    return timbre_df

---

## <a id='CreateSTL'></a> Create STL File

### <a id='BuildDF'></a> Build the Data Frame
Using the pre-built functions to construct the timbre data frame.

In [5]:
# Don't run if already done
if 'sp' not in locals():
    sp = connect_to_spotify()

# Search and retrieve a track URI
track_uri = get_uri(sp)

# Create timbre dataframe
timbre_df = create_timbre_df(track_uri)

Enter your Spotify Client ID:  ed9307841d3542df8819aec9a4f0ec84
Enter your Spotify Secret Token:  8208717955574be6a24163ed59675094


Connected to Spotify!

Retrieving Track URI.


Input artist name:  Chris Stapleton
Input track name:  Tennessee Whiskey



Track found!

Track name: Tennessee Whiskey
Artist name: Chris Stapleton
Track URI: spotify:track:3fqwjXwUGN6vbzIwvyFMhx


In [6]:
# Check 
timbre_df.head()

,start_time,loudness,timbre,timbre_num,timbre_log,timbre_scaled,loudness_scaled,timbre_log_w_loud,timbre_scaled_w_loud,timbre_X,timbre_Y
0,0.0,-60.0,0.000,1,2.602060,1.445857,1.0,2.602060,1.445857,2.253,1.301
1,0.0,-60.0,171.130,2,2.756735,1.812343,1.0,2.756735,1.812343,1.378,2.387
2,0.0,-60.0,9.469,3,2.612221,1.466135,1.0,2.612221,1.466135,0.000,2.612
3,0.0,-60.0,-28.480,4,2.569982,1.384865,1.0,2.569982,1.384865,-1.285,2.226
4,0.0,-60.0,57.491,5,2.660383,1.568978,1.0,2.660383,1.568978,-2.304,1.330


<br>

### <a id='SetXYZ'></a> Set X, Y, and Z Variables
From [Creating a 3D Audio Model](https://nbviewer.org/github/JonYarber/music_modeling/blob/main/python/04Creatinga3DAudioModel.ipynb) notebook, generate the two-dimensional X, Y, and Z arrays.

In [8]:
# Store the number of segments in the song for our reshapes
num_segments = len(timbre_df.start_time.unique())

X = timbre_df.timbre_X.values.reshape(num_segments, 13).T

Y = timbre_df.timbre_Y.values.reshape(num_segments, 13).T

Z = timbre_df.start_time.values.reshape(num_segments, 13).T

print(f'Shape of X: {X.shape}')
print(f'Shape of Y: {Y.shape}')
print(f'Shape of Z: {Z.shape}')

Shape of X: (13, 866)
Shape of Y: (13, 866)
Shape of Z: (13, 866)


<br>

### <a id='ExtractSTL'></a> Extract the STL file
This task ultimately comes down to just three lines of code. However, it was by far the most challenging part of the project. At one point, I honestly believed it was impossible to create a 3D print file using data from Python, as there is very little literature on how to accomplish this. While I successfully generated what I needed, I cannot say with complete certainty that the same approach would work for other models. For this one, it ultimately required having the data in the correct format and a thorough understanding of the <a href = 'https://pyvista.org/'><b>PyVista</b></a> library.

In [9]:
import pyvista as pv

# Create Structured Grid
mesh = pv.StructuredGrid(X, Y, Z)

# Convert to PolyData
poly_data = mesh.extract_surface()

# Extract to .stl
poly_data.save('song_polydata.stl')

----

## <a id='PrintSTL'></a> Printing the STL
Unfortunately, this is where this part of the project came to a grinding halt. <br>
When I imported the model into the 3D printing software, it appeared extremely disproportionate. I estimated its dimensions to be around 2 inches in diameter and 6 feet long. My printer couldn’t handle that scale, and even if it could, the model would look awkward. I spent considerable time tweaking parameters, but without success. I also attempted to adjust the original model, but that didn't help either.<br>
Ultimately, I realized that to create a comprehensible, reasonable, and aesthetically pleasing cylindrical shape, the model needs to be quite long. While it can be 'squished,' it doesn’t look good and results in ridges that the 3D printer can't accommodate.<br>
I will soon include some screenshots of what the model looked like in the 3D printing software. The STL file can be found in the project repository. If anyone has ideas or suggestions for fixing this issue, please reach out to me on GitHub.

[Back to top](#top)